In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy.random as np
import sys
import matplotlib

%matplotlib inline

Import các thư viện cần thiết

In [ ]:
print('Python version ' + sys.version)
print('Pandas version: ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Xem các version

In [ ]:
# set seed
np.seed(111)

# Tạo dữ liệu thử nghiệm
def CreateDataSet(Number=1):
    
    Output = []
    
    for i in range(Number):
        
        # tạo phạm vi thứ 2 hàng tuần
        rng = pd.date_range(start='1/1/2009', end='12/31/2012', freq='W-MON')
        
        #Tạo dữ liệu ngẫu nhiên
        data = np.randint(low=25,high=1000,size=len(rng))
        
        # Nhóm trạng thái
        status = [1,2,3]
        
        # Danh sách trạng thái ngẫu nhiên
        random_status = [status[np.randint(low=0,high=len(status))] for i in range(len(rng))]
        
        # State pool
        states = ['GA','FL','fl','NY','NJ','TX']
        
        # Danh sách trạng thái ngẫu nhiên 
        random_states = [states[np.randint(low=0,high=len(states))] for i in range(len(rng))]
    
        Output.extend(zip(random_states, random_status, data, rng))
        
    return Output

Tạo dữ liệu

In [ ]:
dataset = CreateDataSet(4)
df = pd.DataFrame(data=dataset, columns=['State','Status','CustomerCount','StatusDate'])
df.info()

Tạo dữ liệu đưa vào khung

In [ ]:
df.head()

Xuất dữ liệu

In [ ]:
# Save results to excel
df.to_excel('Lesson3.xlsx', index=False)
print('Done')

Done


Save dữ liệu vào file excel

In [ ]:
pd.read_excel?

hướng dẫn hàm đọc dữ liệu 

In [ ]:
# Location of file
Location = r'/content/Lesson3.xlsx'

# Parse a specific sheet
df = pd.read_excel(Location, 0, index_col='StatusDate')
df.dtypes

lấy file dữ liệu excel 

In [ ]:
df.index

đọc dữ liệu

In [ ]:
df.head()

đọc dữ liệu 

In [ ]:
df['State'].unique()


giá trị trạng thái

In [ ]:
#Chuyển đổi thành chữ hoa và xóa bộ nhớ đệm 
df['State'] = df.State.apply(lambda x: x.upper())

In [ ]:
df['State'].unique()

array(['GA', 'FL', 'TX', 'NY', 'NJ'], dtype=object)

In [ ]:
#chỉ lấy status =1
mask = df['Status'] == 1
df = df[mask]

In [ ]:

mask = df.State == 'NJ'
df['State'][mask] = 'NY'

Chuyển NJ thành NY

In [ ]:
df['State'].unique()

array(['GA', 'FL', 'NY', 'TX'], dtype=object)

In [ ]:
df['CustomerCount'].plot(figsize=(15,5));

Vẽ biểu đồ

In [ ]:
sortdf = df[df['State']=='NY'].sort_index(axis=0)
sortdf.head(10)

Sắp xếp dữ liệu

In [ ]:

Daily = df.reset_index().groupby(['State','StatusDate']).sum()
Daily.head()

Status  CustomerCount
State StatusDate                       
FL    2009-01-12       1            901
      2009-02-02       1            653
      2009-03-23       1            752
      2009-04-06       2           1086
      2009-06-08       1            649

reset index

In [ ]:
del Daily['Status']
Daily.head()

xóa cột Status

In [ ]:

Daily.index

dataframe của index

In [ ]:

Daily.index.levels[0]

Index(['FL', 'GA', 'NY', 'TX'], dtype='object', name='State')

chỉ chọn State

In [ ]:

Daily.index.levels[1]

chỉ chọn StatusDate 

In [ ]:
Daily.loc['FL'].plot()
Daily.loc['GA'].plot()
Daily.loc['NY'].plot()
Daily.loc['TX'].plot();

Biểu đồ theo cột trạng thái


In [ ]:
Daily.loc['FL']['2012':].plot()
Daily.loc['GA']['2012':].plot()
Daily.loc['NY']['2012':].plot()
Daily.loc['TX']['2012':].plot();

Vẽ biểu đồ năm 2012

In [ ]:
# Calculate Outliers
StateYearMonth = Daily.groupby([Daily.index.get_level_values(0), Daily.index.get_level_values(1).year, Daily.index.get_level_values(1).month])
Daily['Lower'] = StateYearMonth['CustomerCount'].transform( lambda x: x.quantile(q=.25) - (1.5*x.quantile(q=.75)-x.quantile(q=.25)) )
Daily['Upper'] = StateYearMonth['CustomerCount'].transform( lambda x: x.quantile(q=.75) + (1.5*x.quantile(q=.75)-x.quantile(q=.25)) )
Daily['Outlier'] = (Daily['CustomerCount'] < Daily['Lower']) | (Daily['CustomerCount'] > Daily['Upper']) 

# Remove Outliers
Daily = Daily[Daily['Outlier'] == False]

In [ ]:
Daily.head()

CustomerCount  Lower   Upper  Outlier
State StatusDate                                       
FL    2009-01-12            901  450.5  1351.5    False
      2009-02-02            653  326.5   979.5    False
      2009-03-23            752  376.0  1128.0    False
      2009-04-06           1086  543.0  1629.0    False
      2009-06-08            649  324.5   973.5    False

In [ ]:
# Combine all markets

# Get the max customer count by Date
ALL = pd.DataFrame(Daily['CustomerCount'].groupby(Daily.index.get_level_values(1)).sum())
ALL.columns = ['CustomerCount'] # rename column

# Group by Year and Month
YearMonth = ALL.groupby([lambda x: x.year, lambda x: x.month])

# What is the max customer count per Year and Month
ALL['Max'] = YearMonth['CustomerCount'].transform(lambda x: x.max())
ALL.head()

,CustomerCount,Max
StatusDate,,
2009-01-05,877,901
2009-01-12,901,901
2009-01-19,522,901
2009-02-02,953,953
2009-02-23,710,953


In [ ]:
pd.date_range?

In [ ]:

data = [1000,2000,3000]
idx = pd.date_range(start='12/31/2011', end='12/31/2013', freq='A')
BHAG = pd.DataFrame(data, index=idx, columns=['BHAG'])
BHAG

Tạo dataframe BHAG

In [ ]:

combined = pd.concat([ALL,BHAG], axis=0)
combined = combined.sort_index(axis=0)
combined.tail()

Kết hợp các khung dữ liệu

In [ ]:
fig, axes = plt.subplots(figsize=(12, 7))

combined['BHAG'].fillna(method='pad').plot(color='green', label='BHAG')
combined['Max'].plot(color='blue', label='All Markets')
plt.legend(loc='best');

Vẽ biểu đồ

In [ ]:
# Group by Year and then get the max value per year
Year = combined.groupby(lambda x: x.year).max()
Year

Nhóm khung dữ liệu theo năm và khách hàng tối đa của năm đó

In [ ]:
# Add a column representing the percent change per year
Year['YR_PCT_Change'] = Year['Max'].pct_change(periods=1)
Year

,CustomerCount,Max,BHAG,YR_PCT_Change
2009,2452.0,2452.0,NaN,NaN
2010,2065.0,2065.0,NaN,-0.157830
2011,2711.0,2711.0,1000.0,0.312833
2012,2061.0,2061.0,2000.0,-0.239764
2013,NaN,NaN,3000.0,0.000000


In [ ]:
(1 + Year.loc[2012,'YR_PCT_Change']) * Year.loc[2012,'Max']

1566.8465510881595

Dự báo số lượng khách hàng cuối năm tới

In [ ]:
# First Graph
ALL['Max'].plot(figsize=(10, 5));plt.title('ALL Markets')

# Last four Graphs
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
fig.subplots_adjust(hspace=1.0) ## Create space between plots

Daily.loc['FL']['CustomerCount']['2012':].fillna(method='pad').plot(ax=axes[0,0])
Daily.loc['GA']['CustomerCount']['2012':].fillna(method='pad').plot(ax=axes[0,1]) 
Daily.loc['TX']['CustomerCount']['2012':].fillna(method='pad').plot(ax=axes[1,0]) 
Daily.loc['NY']['CustomerCount']['2012':].fillna(method='pad').plot(ax=axes[1,1]) 

# Add titles
axes[0,0].set_title('Florida')
axes[0,1].set_title('Georgia')
axes[1,0].set_title('Texas')
axes[1,1].set_title('North East');

Vẽ biểu đồ